# Benchmarking Aproximate Nearest Neighbors

# About

In order for embedding retrieval to work at scale, need to use a vector database.
We also need to use Approximate Nearest Search instead of brute force.


In this notebook, we will use [FAISS]() a library from facebook.

We will compare a brute force and the speedup gained from `IVF`.

For a more detailed comparision, take a look <a href="http://ann-benchmarks.com/">here</a> to find other solutions and benchmark data.


We will look at `performance` and `recall@1`

# Setup

In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
import faiss
import datasets

## Load the embeddings of the image corpus

In [2]:
dset = datasets.load_from_disk("../data/processed_embeddings")
## these embeddings will be used to create the search space.
corpus = dset['embeddings']


corpus = np.array(corpus).astype('float32')
corpus = np.unique(corpus, axis=0)

In [3]:
corpus.shape

(24954, 512)

In [4]:
corpus

array([[-0.08344752,  0.01604629,  0.03037108, ...,  0.03962855,
        -0.02023211, -0.01102281],
       [-0.07890625,  0.02533851,  0.00522987, ...,  0.02622218,
        -0.05418065, -0.00765004],
       [-0.0781679 ,  0.03937826, -0.01087696, ...,  0.04282334,
        -0.02091636, -0.01027698],
       ...,
       [ 0.0878398 ,  0.01232621,  0.00077178, ..., -0.00705758,
         0.01574707, -0.01541145],
       [ 0.0882502 ,  0.03615745, -0.00961868, ...,  0.01392467,
         0.00077467, -0.02139922],
       [ 0.09195283,  0.04004925, -0.00255262, ...,  0.0036222 ,
        -0.0181689 , -0.04212729]], dtype=float32)

In [5]:
dimension = corpus.shape[-1]
dimension

512

# Flat Index / Brute Force


FAISS supports a bruteforce index.    
This index is good if you want perfect recall.   
It requires all the data to be fit in memory.  

## Create the index

In [6]:
x_corpus = corpus
x_corpus.shape
dimension = x_corpus.shape[-1]

initialize the flat index for data dimension.    
In current example it is 512


In [7]:
index = faiss.IndexFlatL2(dimension)

since it is a brute force index, there is no "training" or parameters to learn

In [8]:
index.is_trained


True

add data to the index. This is a CPU based index.

In [9]:
index.add(x_corpus)               

In [10]:
len(x_corpus)

24954

number of vectors / results to retrieve

In [11]:
k =1

#### Index Search
search method returns query indices (I) similar to search query vector and their euclidean distances (D) from the search query vector.

search for single vector and get top 1 result

In [12]:
%%timeit
D, I = index.search(x_corpus[:1], k=1)   

4.38 ms ± 39.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


search for all vectors in corpus and get top 1 result

In [13]:
%%time
D, I = index.search(x_corpus, k=1)     

CPU times: user 30.3 s, sys: 8.92 ms, total: 30.3 s
Wall time: 10.3 s


distance of vector in corpus to query vector

In [14]:
D

array([[0.0000000e+00],
       [0.0000000e+00],
       [3.5762787e-07],
       ...,
       [0.0000000e+00],
       [1.3113022e-06],
       [7.1525574e-07]], dtype=float32)

top vertex id 



In [15]:
I

array([[    0],
       [    1],
       [    2],
       ...,
       [24951],
       [24952],
       [24953]])

because we are using the entire corpus and the ids are sequential, the ideal recall would be sequential too

In [16]:
res = I[:,0] == np.array( list(range(len(x_corpus))))
res

array([ True,  True,  True, ...,  True,  True,  True])

In [17]:
np.where(res == False)

(array([], dtype=int64),)

In [18]:
{
 "recall@1":  res.sum()
 , "num_vectors":  len(res)
 , "mismatch":    len(res) - res.sum()
}


{'recall@1': 24954, 'num_vectors': 24954, 'mismatch': 0}

For this corpus, we are able to find the query vector as position 1

# FAISS IVF

<img src="https://d33wubrfki0l68.cloudfront.net/44acb1425f25e30ca058daec92bdb209c6c47ad2/e92fc/images/faiss5.png" width="500"/>

<p> Image from Pinecone Faiss Tutorial </p>
https://www.pinecone.io/learn/faiss-tutorial/


**Parameters**:
- nlist : number of clusters
- nprobe: number of clusters to search

In [19]:
nlist = 20 # number of clusters
quantizer = faiss.IndexFlatL2(dimension)  # the other index
index = faiss.IndexIVFFlat(quantizer, dimension, nlist, faiss.METRIC_L2)

In [20]:
assert not index.is_trained
index.train(x_corpus)
assert index.is_trained

In [21]:
index.add(x_corpus)         

we need to train the index first with a sample of vectors before indexing

search for single vector

In [22]:
%%timeit

index.nprobe = 1              # default nprobe is 1

D, I = index.search(x_corpus[:1], k)     # actual search

114 µs ± 729 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


in the above, we are only querying 1/20 of the search space

In [23]:
%%timeit


index.nprobe = 10              # default nprobe is 1

D, I = index.search(x_corpus[:1], k)     # actual search

1.55 ms ± 13.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


in the above, we are only querying half of the search space

In [24]:
%%timeit


index.nprobe = 20              # default nprobe is 1

D, I = index.search(x_corpus[:1], k)     # actual search

4.88 ms ± 57.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


in the above, we are querying the entire search space. This is the same as using Brute Force.

search for entire corpus

In [25]:
%%time


index.nprobe = 1              

D, I = index.search(x_corpus, k)     # actual search

CPU times: user 8.18 s, sys: 23.5 ms, total: 8.21 s
Wall time: 1.05 s


In [26]:
z = I[:,0] == np.array( list(range(len(x_corpus))))
{
 "recall@1":  z.sum()
 , "num_vectors":  len(z)
 , "mismatch":    len(z) - z.sum()
}


{'recall@1': 24954, 'num_vectors': 24954, 'mismatch': 0}

increase the number of cells that are probed

In [27]:
%%timeit

index.nprobe = 5              # default nprobe is 1

D, I = index.search(x_corpus, k)    

4.73 s ± 230 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [28]:
z = I[:,0] == np.array( list(range(len(x_corpus))))
{
 "recall@1":  z.sum()
 , "num_vectors":  len(z)
 , "mismatch":    len(z) - z.sum()
}


{'recall@1': 24954, 'num_vectors': 24954, 'mismatch': 0}